# Init ptbxl

In [1]:
import numpy as np
import wfdb
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels as stats
import os

In [ ]:
path = '../data/physionet.org/files/ptb-xl/1.0.3/ptbxl_database.csv'
df = pd.read_csv(path)
df.head()


# Ziel Variable

In [4]:
df.columns

Index(['ecg_id', 'patient_id', 'age', 'sex', 'height', 'weight', 'nurse',
       'site', 'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr'],
      dtype='object')

In [11]:
print(f'Die Zielvariable hat {len(df["scp_codes"].unique())} unique values')

Die Zielvariable hat 5463 unique values


In [12]:
df['scp_codes'].head()

0    {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
1                {'NORM': 80.0, 'SBRAD': 0.0}
2                  {'NORM': 100.0, 'SR': 0.0}
3                  {'NORM': 100.0, 'SR': 0.0}
4                  {'NORM': 100.0, 'SR': 0.0}
Name: scp_codes, dtype: object

Die Zielvariable hat 5463 eindeutige Werte. Das liegt daran, dass sie in form von Dictionary ist (aber als String gespeichert ist), in dem viele verschiedene SCP-Codes enthalten sein können.

In [36]:
mi = ["IMI", "ASMI", "ILMI", "AMI", "ALMI", "INJAS", "LMI", "INJAL", "IPLMI", "IPMI", "INJIN", "INJLA", "PMI", "INJIL"]

In [38]:
counter = 0
for element in mi:
    for value in df['scp_codes']:
        if element in value:
            counter += 1

print(counter)

7713
